<a href="https://colab.research.google.com/github/Arpit1118/LLM/blob/main/MiniGPT_Failed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

# Hyperparameters
block_size = 8  # context window
batch_size = 16
embedding_dim = 32
num_heads = 2
num_layers = 2
learning_rate = 0.0001
num_epochs = 400

# Load text
with open("tiny_dataset.txt", "r") as f:
    text = f.read()

# Tokenization
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

# Train/val split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    d = train_data if split == "train" else val_data
    ix = torch.randint(len(d) - block_size, (batch_size,))
    x = torch.stack([d[i:i+block_size] for i in ix])
    y = torch.stack([d[i+1:i+block_size+1] for i in ix])
    return x, y

# Model
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = nn.Embedding(block_size, embedding_dim)
        self.transformer_blocks = nn.Sequential(*[
            nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, batch_first=True)
            for _ in range(num_layers)
        ])
        self.lm_head = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        B, T = x.shape
        tok_emb = self.token_embedding(x)
        pos_emb = self.pos_embedding(torch.arange(T, device=x.device))
        x = tok_emb + pos_emb
        x = self.transformer_blocks(x)
        logits = self.lm_head(x)
        return logits

model = GPT()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    xb, yb = get_batch("train")
    logits = model(xb)
    loss = F.cross_entropy(logits.view(-1, vocab_size), yb.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Text Generation
def generate(idx, max_new_tokens=50):
    model.eval()
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits = model(idx_cond)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

# Start with "T"
context = torch.tensor([[stoi['T']]], dtype=torch.long)
output = generate(context, max_new_tokens=100)
print(decode(output[0].tolist()))


Epoch 0, Loss: 3.6815
Epoch 10, Loss: 3.4729
Epoch 20, Loss: 3.3066
Epoch 30, Loss: 3.1964
Epoch 40, Loss: 3.3391
Epoch 50, Loss: 3.0885
Epoch 60, Loss: 3.1379
Epoch 70, Loss: 2.9660
Epoch 80, Loss: 2.9482
Epoch 90, Loss: 2.9930
Epoch 100, Loss: 2.8843
Epoch 110, Loss: 2.8407
Epoch 120, Loss: 2.8068
Epoch 130, Loss: 2.8014
Epoch 140, Loss: 2.5859
Epoch 150, Loss: 2.6082
Epoch 160, Loss: 2.6740
Epoch 170, Loss: 2.4984
Epoch 180, Loss: 2.5507
Epoch 190, Loss: 2.4869
Epoch 200, Loss: 2.5444
Epoch 210, Loss: 2.4294
Epoch 220, Loss: 2.3008
Epoch 230, Loss: 2.5340
Epoch 240, Loss: 2.3988
Epoch 250, Loss: 2.4055
Epoch 260, Loss: 2.4118
Epoch 270, Loss: 2.2378
Epoch 280, Loss: 2.2837
Epoch 290, Loss: 2.3419
Epoch 300, Loss: 2.3738
Epoch 310, Loss: 2.1143
Epoch 320, Loss: 2.3671
Epoch 330, Loss: 2.0733
Epoch 340, Loss: 2.3203
Epoch 350, Loss: 2.2965
Epoch 360, Loss: 2.1971
Epoch 370, Loss: 2.0292
Epoch 380, Loss: 2.2302
Epoch 390, Loss: 2.2048
Tu"=mS.
D.tvaYoy is vayaxl vDp awLY .wwaou_)aob.das